### Business Overview

Oxford dictionry descrives an echo chamber as, "an environment in which somebody encounters only opinions and beliefs similar to their own, and does not have to consider alternatives." According to this Pe Study https://www.pewresearch.org/journalism/2021/01/12/news-use-across-social-media-platforms-in-2020/, 53% of American Adults say they get news from social media “often” or “sometimes,”. And, this https://www.pnas.org/content/118/9/e2023301118 2021 National Academy of Sciences study on echo chambers on social media platforms found "the aggregation of users in homophilic clusters dominate online interactions on Facebook and Twitter." While social media sites need to take some of the responsibility, I want to take a look at the content being shared and discussed within the echo chambers. It is important to take a look at what information and messaging is being spread with the echo chambers. An article in Scientific American https://www.scientificamerican.com/article/why-social-media-makes-us-more-polarized-and-how-to-fix-it/ discusses how social media, specifically social media influencers, have a disproportionate impact on their communities. I am not investigating how these influencers came to be. I am interested in what they are sharing in the space.

The good people at Allsides.com try to help people "easily identify different perspectives so you can get the full picture and think for yourself." Today's media is very biased. And a lot of media sources are very acessible. As shown in National Academy of Sciences Paper, newsfeeds are often an echo chamber based on thier history of related articles and newsites, especially on Facebook. By nature, echo chambers remove media sources outside of someone's chamber, making it difficult to understand different view points regarding news events.

A Stanford article https://news.stanford.edu/2019/08/22/the-power-of-language-how-words-shape-people-culture/ points out "Studying how people use language – what words and phrases they unconsciously choose and combine – can help us better understand ourselves and why we behave the way we do." By paying attention to the words used in a article, the reader can critically think about what the article is presenting. Know what words and terms determine bias creates trasparency about topics in the article. The reader can know certain words and phrases are used in different biased articles and go deeper into the article to think about why the author used the words they did, or what message are they sending by using that word. Helping people become aware of what words and phrases drive bias in news media content can help the people understand the both sides of an argument.

People who do want to explore both side of an argument may visit a site like AllSides.com to guage a news site's bias. Allsides uses many methods to rate bias, including third party data with the requirement of transparency in the system. I have developed a model to determine bias in a corpus of news articles based on the laguage used in each article. To add transparency and explainability to my model I have utilized LIME to show what words and phrases leading to a better understanding of media bias and more informed readers

### Data Understanding

Source: I would have liked to use one dataset, however, the computational cost was too high which lead me to shrinking the size of the dataset. This data was gathered from https://components.one/datasets/all-the-news-2-news-articles-dataset/ and https://www.kaggle.com/snapcrack/all-the-news. The Kaggle dataset was created from the original source site but different for a different database. The DataFrames contained the same content and allowed me to use information from both to develop the working DataFrame.

I began by looking at the data and understanding I would only be able to predict left, center or right. I did not have enough data to make a 5 class predictor. From there I decided to select two left foused, two center focused and two right focused media outlets. The news sources were determined by looking at MediaBias/Fact Check and Allsides.com Media Bias lists and selecting sources that each independent side agreed on. After selecting my sources, I began to gather the articles in one DataFrame.

I started to gather information for my Dataframe in the WorkingDF noteboook. I decided to go with two sources for each category of bias. The sources do not have an equal number of articles. And each bias is not an equal split. The Kaggle Dataset had all the sources I needed for the right leaning articles and one source for the center and left biases. I imported each csv file from Kaggle and selected the sources I needed. I then flipped over to the split notebook. Due to the size of the dataset, I randomly selected 35 percent of the 2.7 million articles in the compnents dataset. I then to pulled stories from Vox and The Hill. To maintain class balance, I randomly select between 16,000 and 17,000 texts pulished on The Hill's website. I then pickled the two dataframes and imported them into the WorkingDF notebook. I then combined all of the dataframes into one woring DataFrame. I then pickled my working DataFrame and imported into the EDA/preprocessing notebook. Once I processed the data for modeling I again pickled the processed DataFrame and loaded it into the Modeling notebook. For this notebook I have pulled the unprocessed Workiong DataFrame to elaborate on my preprocessing steps. In total, I gathered approximately 85,000 articles from from the two datasets. Each row contains an article, publisher and bias label. For modeling I will use the article and bias.

This data set could be used to its full potential with better computing power. The article corous also has a limited amount of right leaning news sources. There is an abundance of left leaning sources. With a greater variety of sources, I would like to expand this classifer.

In [2]:
#import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from itertools import islice

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
import re
import string

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical

import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from textblob import TextBlob

In [ ]:
def countplot(df, col, hue=None, rotation=None, order=None):
    """
    This function builds a Seaborn countplot and allows the user to set certain parameters to customize the graph.
    
    df - dataframe being used
    col- column in dataframe being used
    hue - second column being used(if any)
    rotation - sets the rotation of the x-ticks for readability
    """
    fig, ax = plt.subplots(figsize=(12,8))
    sns.countplot(data = df, x = col, hue = hue, order = order)
    ax.set_xticklabels(labels = ax.get_xticklabels(), rotation= rotation, fontsize = 15)
    ax.set_xlabel(xlabel = col, fontsize = 20)
    ax.tick_params(axis='y', which='major', labelsize=15)
    ax.set_ylabel(ylabel = "Number of articles", fontsize = 20)
    ax.set_title(f"Number of articles per {col}", fontsize = 30)
    plt.show()
    
def evaluate(model, X_tr, y_tr, X_te, y_te):
    print('Accuracy Score:')
    print(f'Train - {accuracy_score(y_tr, model.predict(X_tr))}')
    print(f'Test - {accuracy_score(y_te, model.predict(X_te))}')
    print('  ')
    print('Classification Report:')
    print(classification_report(y_te, model.predict(X_te)))
    print('Confusion matrix for test data')
    return plot_confusion_matrix(model, 
                                 X_te, y_te, 
                                 include_values=True, 
                                 display_labels =  ["Left", "Center", "Right"], 
                                 cmap=plt.cm.Blues)

def create_embedding_matrix(glove_filepath, word_index, embedding_dim):
    '''
    Grabs the embeddings just for the words in our vocabulary
    
    Inputs:
    glove_filepath - string, location of the glove text file to use
    word_index - word_index attribute from the keras tokenizer
    embedding_dim - int, number of dimensions to embed, a hyperparameter
    
    Output:
    embedding_matrix - numpy array of embeddings
    '''
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(glove_filepath, encoding="UTF-8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
#take engineered dataframe from pickle 
df = pd.read_pickle("../data/df.pkl")
df.head()

In [ ]:
# taking a look at  the legnths of each article
df['word_count'] = df["Content"].apply(lambda x: len(nltk.word_tokenize(str(x))) )
df['char_count'] = df["Content"].apply(lambda x: sum(len(word) for word in nltk.word_tokenize(str(x))) )
df['sentence_count'] = df["Content"].apply(lambda x: len(nltk.sent_tokenize(str(x))) )

I limit the articles to at least 50 words and no more than 5000 words. Limiting the corpus cleans out content willed with meaningless content sunch as random characters or advertsiments and larger contents that are filled with a stream of breaking news updates rather than written articles. There can be another project in investigating live tweets of a news story.

In [ ]:
df = df[(df["word_count"] < 5000) & (df["word_count"] > 50)]
df

In [ ]:
#check the shape of the data
df.shape

In [ ]:
#is anything null?
df.isna().sum()

In [ ]:
#How many atcles per publication?
df["Publication"].value_counts()

In [ ]:
#How many atcles per Bias?
df["Bias"].value_counts()

In [ ]:
print("Percentage of Articles per Bias in Dataset:")
df["Bias"].value_counts(normalize=True)*100

## Visualize the data

In [ ]:
#number of articles per publication
countplot(df, "Publication", order = df["Publication"].value_counts().index)

In [ ]:
#number of articles per publication
countplot(df, "Bias", order = ["Left", "Center", "Right"])

In [ ]:
#group 
df_plot = df.groupby(['Publication', 'Bias']).size().reset_index().pivot(columns='Publication', index='Bias', values=0)

In [ ]:
df_plot = df_plot.reindex(["Left", "Center", "Right"])

In [ ]:
df_plot.plot(kind='bar', stacked=True, figsize=(20,10))
plt.xticks(rotation="horizontal", fontsize = 30)
plt.xlabel(xlabel = "Bias", fontsize = 40)
plt.yticks(rotation="horizontal", fontsize = 30)
plt.ylabel(ylabel = "Number of Articles", fontsize = 40)
plt.title(label= "Number of Articles per Publication per Bias", fontsize = 40)
plt.legend(bbox_to_anchor=(1.3, 1), loc="upper right", ncol=1, fontsize= 25, title = "Publication", title_fontsize= 35)
plt.show()

In [ ]:
#take a look at the distribution of number of words in each article
fig, ax = plt.subplots(figsize=(12,8))
hp = sns.histplot(data = df, x="word_count", hue = "Bias", kde=True)
ax.set_xlabel(xlabel = "Number of words", fontsize = 20)
ax.tick_params(axis='y', which='major', labelsize=15)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.set_ylabel(ylabel = "Number of articles", fontsize = 20)
ax.set_title(f"Number of words per article", fontsize = 30)
plt.show()

In [ ]:
#take a look at the distribution of number of characters in each article
fig, ax = plt.subplots(figsize=(12,8))
sns.histplot(data = df, x="char_count", hue = "Bias")
ax.set_xlabel(xlabel = "Number of words", fontsize = 20)
ax.tick_params(axis='y', which='major', labelsize=15)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.set_ylabel(ylabel = "Number of articles", fontsize = 20)
ax.set_title(f"Number of characters per article", fontsize = 30)
plt.show()

In [ ]:
#take a look at the distribution of number of sentences in each article
fig, ax = plt.subplots(figsize=(12,8))
sns.histplot(data = df, x="sentence_count", hue = "Bias", bins = 100, kde = True)
ax.set_xlabel(xlabel = "Number of words", fontsize = 20)
ax.tick_params(axis='y', which='major', labelsize=15)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.set_ylabel(ylabel = "Number of articles", fontsize = 20)
ax.set_title(f"Number of sentences per article", fontsize = 30)
plt.show()

## Data Prep

### Preprocessing Data

I went through a series of operations to preprocess the data for Machine Learning. The first step was to encode the bias with a -1 for left, 0 for center and 1 for right. <br>

In the preprocessing step of the project I followed these steps:

- Lower case each word
- Remove unwanted noise from the article(such as Twitter mentions, web adresses and other content)
- Remove all numbers
- Tokenize the list
- Remove common stop words and punctuation
- Lemmatizes each word
- Returns a string of the remaining lemmatized words

Fist I lowercase all the words to simplfy cleaning process. I then remove noice like mentions(@abc1123), hastags(#), and urls because they do not add anything to analysis. I also remove information at the end of The Hill's articles because it is contact information and does not help with analysis. Numbers are removed because they have no context by themselves. We tokenize the list and remove any common stopwords and punctuation. Tokenizing a list is breaking each element in the list into its own list object. Next, I lemmatized the remaining words and combines the tokens back into a string to be vectorized.

In [ ]:
#check the data set
df

In [ ]:
# Multiclass the bias column
dict_sent = {'Left':-1, 
             'Center':0,
             'Right':1}
df["num_bias"] = df["Bias"].map(dict_sent)

Check all three labels

In [ ]:
df.head()

In [ ]:
df.iloc[45000:45005]

In [ ]:
df.tail()

In [ ]:
#initialize the Tokenizer and Lemmatizer
tokenizer = RegexpTokenizer(r"(?u)\b\w\w+\b")
lemma = WordNetLemmatizer()

In [ ]:
# Create a list of new sources to add to stopwords list
source = list(df["Publication"].unique())
source = (map(lambda x: x.lower(), source))
source = list(source)
source

In [ ]:
#create stopwords_list, adds punctuation and the news sources to list
stopwords_list = stopwords.words("english")
stopwords_list += string.punctuation
stopwords_list += source
print(stopwords_list)

In [ ]:
#Regex expressions to remove unwanted noise
pattern_mentions = "((@[a-zA-Z0-9_-]+)((?=\s+)|$|[!?.,-]))"
pattern_views =  "((the views expressed by)([\s\S]*)$)"
pattern_thread = "((view the discussion thread.)([\s\S]*)$)"
pattern_url = "(http\S+)"
pattern_num = "([\d.])"
pattern_pic = "((pic.)?twitter\.com\/[A-Za-z0-9_]{5,1000}(\?(\w+=\w+&?)*)?)"

In [ ]:
#random article in the corpus will use to check the preprocessing step
test = df["Content"].iloc[random.randint(0,df.shape[0])]
test

In [ ]:
#lowercase text
test = test.lower()
test

In [ ]:
# Removes Twitter mentions
test = re.sub(pattern_mentions,"", test)
test

The Hill articles end with "the views expressed by" or "view the discussion thread" both are followed by some contact information. I decided to remove these elements to prevent mis weighing words later in the Vectorizing stage of the project.

In [ ]:
# remove the end of The Hill articles starting at the phrase "the views expressed by"
test =  re.sub(pattern_views,"", test)
test

In [ ]:
# remove the end of The Hill articles starting at the phrase "view the discussion thread"
test =  re.sub(pattern_thread,"", test)
test

In [ ]:
#removes hyperlinks starting with http
test =  re.sub(pattern_url,"", test)
test

In [ ]:
#removes numbers from the text.
test =  re.sub(pattern_num,"", test)
test

In [ ]:
#removes pic.twitter urls
test =  re.sub(pattern_pic,"", test)
test

In [ ]:
#tokenize text
test = tokenizer.tokenize(test)
test[:20]

In [ ]:
#remove stopwords
stopwords_removed=[token for token in test if token not in stopwords_list]
stopwords_removed[:20]

In [ ]:
#lemmatize the remaing words
lemma_list = [lemma.lemmatize(token) for token in stopwords_removed]
lemma_list[:20]

In [ ]:
#sanity check
test = df["Content"].iloc[random.randint(0,df.shape[0])]
test

In [ ]:
preprocess(test)

### Vectorizing data

Once there is a string of processsed data. I use a Term Frequency-Inverse Document Frequency (TFIDF) vectorizer to vectorize the documents. The TF-IDF Vectorizer weight is found by measuring the Term Frequency and Inverse Document Frequency. Term frequency is how many times a term appears in the docuemnt divded my the number of terms in document. The Inverse Document Frequency is the log loss of the total number of documents divided by the number of documents with the term in it. It is based on the idea that rare words contain more information about the content of a document than words that are used many times throughout all the documents. Now our corpus of articles is ready for modeling

In [ ]:
#initialize and fit the count vectorizer
cvec = CountVectorizer(stop_words='english', ngram_range=(1,2), max_features=2000, max_df=.5, min_df= 10)
cvec.fit(X_train["processed_content"])

In [ ]:
#initialize, fit and transform the data
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocess, max_features= 2000, 
                            ngram_range= (1,2),
                            max_df = .5, min_df = 10))

tfidf_vectorizer.fit(X_train["Content"])
X_train_tfidf_vec = tfidf_vectorizer.transform(X_train["Content"])
X_val_tfidf_vec = tfidf_vectorizer.transform(X_val["Content"])
#returna a dataframe using the count vectorizer to label to the columns rather than usless numbers.
X_train_tfidf_vec_df = pd.DataFrame(X_train_tfidf_vec.toarray(),columns=cvec.get_feature_names(), 
                              index=X_train.index)
X_val_tfidf_vec_df = pd.DataFrame(X_val_tfidf_vec.toarray(), columns=cvec.get_feature_names(), 
                              index=X_val.index)

X_train_tfidf_vec_df.shape

In [ ]:
#What the dataframe will look like for modeling
X_train_tfidf_vec_df.head()

In [ ]:
X_val_tfidf_vec_df.head()

## Modeling

For this project I explored many different models. To keep things short in this notebook I have showcased a baseline, simple and 3 models followed by my final model and evaluation. You can find the rest of the models along with tuning in the Modeling Notebook in the Notebooks folder.

### Baseline Model

In [ ]:
#If we were to randomly guess bias 
y_train.value_counts(normalize= True)

### Simple Model

In [ ]:
#set up pipeline
lr = Pipeline(steps=[
    ('vect', TfidfVectorizer('vect', TfidfVectorizer(preprocessor=preprocess, max_features= 2000, 
                            ngram_range= (1,2)))),
    ('clf', LogisticRegression(multi_class="multinomial", random_state=seed))
                                                    ])
#fit to training data
lr.fit(X_train["Content"], y_train)

print(lr.score(X_train["Content"], y_train))
print(lr.score(X_val["Content"], y_val))

### Grid Search/Cross Validate

To save on computational costs in this notebook, I have only included the best performing parameters. All of the parameters explored are in the modeing notbook. I have also used 3 cross validation folds due to the size and cost of running for folds.

#### Logistic Regression

In the grid search, I added Ridge regression to adjust the weights of the coeffecients. I increased the max iterations.I looked at the regularization strength to see how much regression to effect. the solver hyperparameter was limited to 'newton-cg', 'sag', 'saga' and 'lbfgs' for muticlass problems.

In [ ]:
lr_tuned = Pipeline(steps=[
    ('vect', TfidfVectorizer(preprocessor=preprocess, max_features= 2000, 
                            ngram_range= (1,2),
                            max_df = .5, min_df = 10)),
    ('clf', LogisticRegression(multi_class="multinomial", random_state=seed)),
])

param_grid = {
    'clf__penalty': ["l2"],
    'clf__max_iter': [1000],
    'clf__C': [1],
    'clf__solver': ['newton-cg']
}

In [ ]:
grid = GridSearchCV(lr_tuned, param_grid, return_train_score=True)

In [ ]:
output = grid.fit(X_train["Content"], y_train)

In [ ]:
best_lr = output.best_estimator_
best_lr

In [ ]:
evaluate(best_lr, X_train["Content"], y_train, X_val["Content"], y_val)

### LIME
It is important to develop an accurate model. But just as important is the explainability of the model. As more complex models are used for analysis, the level of interpertability decreases. It is difficult to explain how certain models, like a neural network, work under the hood. Opening the black box of accurate complex models is an important aspect of this project. While I do want to accurately predict bias, I also want to understand what is driving decision making and be able to showcase what words are driving decisions.
### TextBlob
TextBlob is another NLP package. It is built on Stanford's NLTK module. I use TextBlob's Polarity and Subjectivity score to give the reader more insight into the article. Polarity is on a 1 to -1 scale where 1 means positive statement and -1 means a negative statement.Subjectivity is rated on a 0 to 1 scale where 0 means highly objective and 1 means highly subjective.

In [ ]:
#This code was found at https://marcotcr.github.io/lime/tutorials/Lime%20-%20multiclass.html
idx = random.randint(0,X_val.shape[0])
class_names=["Left", "Center", "Right"]
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(str(X_val["Content"][idx]), best_lr.predict_proba, num_features=3, labels=[0, 1, 2])
print(f'Document id: {idx}')
print(f'Predicted bias : {class_names[best_lr.predict(list(X_val["Content"][idx])).reshape(1,-1)[0,0]]}')
print(f'True bias: {class_names[y_val[idx]]}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).subjectivity}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).polarity}')
exp.show_in_notebook(text=X_val["Content"][idx], labels=(0,1,2))

### Decision Tree

For the decision tree I decided to explore the depth of the tree, how many samples in each leaf and how many samples to split at internal node. I adjusted these features because the previous model was very overfit.

In [ ]:
dt_tuned = Pipeline(steps=[
    ('vect', TfidfVectorizer(max_features= 2000, 
                            ngram_range= (1,2),
                            max_df = .5, min_df = 10)),
    ('clf', DecisionTreeClassifier(random_state=seed)),
])

param_grid = {
    'clf__max_depth': [25],
    'clf__min_samples_leaf': [10],
    'clf__min_samples_split': [5],
}

In [ ]:
grid = GridSearchCV(dt_tuned, param_grid, return_train_score=True,cv =3)

In [ ]:
output = grid.fit( X_train["Content"], y_train)

In [ ]:
best_dt= output.best_estimator_
best_dt

In [ ]:
evaluate(output.best_estimator_, X_train["Content"], y_train, X_val["Content"], y_val)

In [ ]:
idx = random.randint(0,X_val.shape[0])
class_names=["Left", "Center", "Right"]
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(str(X_val["Content"][idx]), best_dt.predict_proba, num_features=6, labels=[0, 1, 2])
print(f'Document id: {idx}')
print(f'Predicted bias : {class_names[best_dt.predict(list(X_val["Content"][idx])).reshape(1,-1)[0,0]]}')
print(f'True Bias: {class_names[y_val[idx]]}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).subjectivity}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).polarity}')
exp.show_in_notebook(text=X_val["Content"][idx], labels=(0,1,2))

### XGBoost

For the XGboost Model I explored alot of hyperparameters to try and reduce overfitting, however due to time contraints I had to move on to other tasks. I tried to increase the min_child_weight and max_depth was limited to prevent to prune the tree. The learning_rate was adjusted to optimize the Gradient Descent.N-estimators controlls the number of trees to make for each boosting round. colsample_bytree is to limit how much information the model takes in each boosting round. and alpha and lambda control the Ridge and Lasso regularization.

In [ ]:
xgb_tuned = Pipeline(steps=[
    ('vect', TfidfVectorizer(preprocessor=preprocess, max_features= 2000, 
                            ngram_range= (1,2),
                            max_df = .5, min_df = 10)),
    ('clf', XGBClassifier(random_state=seed, n_jobs = 4))])

param_grid = {
            'clf__min_child_weight': [10],
            'clf__max_depth': [15],
            'clf__learning_rate': [0.1],
            'clf__n_estimators': [100],
            'clf__colsample_bytree': [0.3],
            'clf__reg_alpha': [10],
            'clf__reg_lambda': [2],
}

In [ ]:
grid = GridSearchCV(xgb_tuned, param_grid, return_train_score=True, cv = 3)

In [ ]:
output = grid.fit(X_train["Content"], y_train)

In [ ]:
best_xgb = output.best_estimator_
best_xgb

In [ ]:
evaluate(best_xgb, X_train["Content"], y_train, X_val["Content"], y_val)

In [ ]:
idx = random.randint(0,X_val.shape[0])
class_names=["Left", "Center", "Right"]
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(str(X_val["Content"][idx]), best_xgb.predict_proba, num_features=5, top_labels=3)
print(f'Document id: {idx}')
print(f'Predicted bias : {class_names[best_xgb.predict(list(X_val["Content"][idx])).reshape(1,-1)[0,0]]}')
print(f'True Bias: {class_names[y_val[idx]]}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).subjectivity}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).polarity}')
exp.show_in_notebook(text=X_val["Content"][idx], labels=(0,1,2))

### Neural Network

In [ ]:
#finds the length of the longest article
max_length = max([len(s.split()) for s in X_train["Content"]])
max_length

In [ ]:
#tokenizes and vectoizes text 
tokenizer = keras.preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(X_train["Content"])

X_train_token = tokenizer.texts_to_sequences(X_train["Content"])
X_val_token = tokenizer.texts_to_sequences(X_val["Content"])

In [ ]:
#neeed for dimensions in the neural network
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
#pads the documents to make sure they are all the same length
X_train_processed = keras.preprocessing.sequence.pad_sequences(
    X_train_token, maxlen=max_length, padding='post')
X_val_processed = keras.preprocessing.sequence.pad_sequences(
    X_val_token, maxlen=max_length, padding='post')

In [ ]:
#one-hot encode target column
y_train2 = to_categorical(y_train)
y_val2 = to_categorical(y_val)

#vcheck that target column has been converted
y_val2[0:10]

I will use the GloVE pretrained embedder for this network. The GloVE file to large for Git Hub, can be found here: https://nlp.stanford.edu/projects/glove/

In [ ]:
#sets up the embedding dictionary
embedding_dim = 50
embedding_matrix = create_embedding_matrix('../data/glove.6B.50d.txt',
                                           tokenizer.word_index, 
                                           embedding_dim)

In [ ]:
model = keras.models.Sequential(name="tuned")
#Sets the first layer with the embedding dictionary
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=max_length, 
                           trainable=False))
#dropout half of the nodes randomly
model.add(Dropout(0.5))
#a simple dense layer
model.add(layers.Dense(15, activation='relu'))
# flattening these layers down before connecting to dense layer
model.add(layers.Flatten()) 
#final output layer. using softmax because it is a multiclass problem
model.add(layers.Dense(3, activation='softmax'))
#compile the network
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
#fit the model
history = model.fit(X_train_processed, y_train2,
                    epochs=10,
                    batch_size=100,
                    validation_data=(X_val_processed, y_val2))

In [4]:
#display accuracy and loss scores
visualize_training_results(history)

NameError: name 'visualize_training_results' is not defined

In [ ]:
idx = random.randint(0,X_val.shape[0])
class_names=["Left", "Center", "Right"]
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(str(X_val["Content"][idx]), best_xgb.predict_proba, num_features=5, top_labels=3)
print(f'Document id: {idx}')
print(f'Predicted bias : {class_names[best_xgb.predict(list(X_val["Content"][idx])).reshape(1,-1)[0,0]]}')
print(f'True Bias: {class_names[y_val[idx]]}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).subjectivity}')
print(f'Subjectivity Score: {TextBlob(X_val["Content"][idx]).polarity}')
exp.show_in_notebook(text=X_val["Content"][idx], labels=(0,1,2))